Library imports

In [1]:
from logging import warning
import pandas as pd 
import numpy as np
import re
import nltk
import heapq
import string
# NLP tools
import contractions
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# Machine Learning Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes  import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
# Hold out / Grid / Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
# Metrics 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
# .Config 
warning("ignore")
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rícharde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rícharde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Utilize o arquivo moviesreviews.tsv (sep = ‘\t’).

In [2]:
reviews_df = pd.read_csv(r"C:\Users\Rícharde\Documents\Dell lead\Atividade 06\moviereviews.tsv", sep='\t')

In [3]:
reviews_df.head(10)

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...
5,neg,"to put it bluntly , ed wood would have been pr..."
6,neg,"synopsis : melissa , a mentally-disturbed woma..."
7,neg,tim robbins and martin lawernce team up in thi...
8,neg,"in "" gia "" , angelina jolie plays the titular ..."
9,neg,"in 1990 , the surprise success an unheralded l..."


# Faça pré-processamento dos dados.


DataSet analysys 

In [4]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2000 non-null   object
 1   review  1965 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [5]:
reviews_df.isnull().sum()

label      0
review    35
dtype: int64

In [6]:
reviews_df['label'].value_counts()

neg    1000
pos    1000
Name: label, dtype: int64

In [7]:
# Null values position
# creating bool series True for NaN values 
bool_series = pd.isnull(reviews_df["review"]) 
    
# filtering data 
# displaying data only with Gender = NaN 
reviews_df[bool_series]

,label,review
140,pos,NaN
208,pos,NaN
270,neg,NaN
334,neg,NaN
448,neg,NaN
522,neg,NaN
606,pos,NaN
696,neg,NaN
728,pos,NaN
738,neg,NaN


In [8]:
reviews_df = reviews_df.dropna()
print(reviews_df.isnull().sum())

label     0
review    0
dtype: int64


In [9]:
reviews_df['label'].value_counts()

neg    983
pos    982
Name: label, dtype: int64

Label transform 

In [10]:
# Label transform 
data = LabelBinarizer().fit_transform(reviews_df["label"])
reviews_df["label"] = data

Tokenizing sentences

In [11]:
def Tokenizing_Sentences(Data,colun_term):
    sentences = Data[colun_term]
    Iloc = 0
    for sentece in sentences:
        tonken_text = word_tokenize(sentece,language="english",preserve_line=False)
        Data[colun_term].iloc[Iloc] = tonken_text
        Iloc+=1
    return Data

In [12]:
reviews_df = Tokenizing_Sentences(reviews_df,'review')

C:\Users\Rícharde\AppData\Local\Temp\ipykernel_9936\812245980.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data[colun_term].iloc[Iloc] = tonken_text


Removing Stop Words

In [13]:
stop_Words = set(stopwords.words("english"))

In [14]:
def Verify_Stop_Words(sentence):
    text = list(sentence)
    for word in stop_Words:    
        if text.__contains__(word):
            text.remove(word)
    return text

In [15]:
sentences = reviews_df["review"]
Iloc = 0
for sentence in sentences:
    text = Verify_Stop_Words(sentence)
    reviews_df["review"].iloc[Iloc] = text
    Iloc +=1

C:\Users\Rícharde\AppData\Local\Temp\ipykernel_9936\80236415.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review"].iloc[Iloc] = text


Removing Punctuation and special caracter

In [16]:
sentences = reviews_df["review"]
Iloc = 0                                                                                                                
for sentence in sentences:
    text = re.sub(r'[^\w\s]', '',str(sentence))
    reviews_df["review"].iloc[Iloc] = text
    Iloc +=1

C:\Users\Rícharde\AppData\Local\Temp\ipykernel_9936\1056579119.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review"].iloc[Iloc] = text


In [17]:
reviews_df

,label,review
0,0,films like mouse hunt get theatres nt law som...
1,0,talented actresses blessed demonstrated wide a...
2,1,extraordinary year australian films shine h...
3,1,according hollywood movies made last decades ...
4,0,first press screening 1998 already ve gotten p...
...,...,...
1995,1,like movies albert brooks i really like movie...
1996,1,might surprise know joel ethan coen brought u...
1997,1,verdict spinechilling drama horror maestro st...
1998,1,want correct i wrote former retrospective davi...


# Separe os dados em treino e teste.

In [18]:
# Data separation
X = reviews_df["review"]
y = reviews_df["label"]

# extração de características do texto utilizando BOW e TF-IDF

Word Count 

In [19]:
count_vec = CountVectorizer(ngram_range=(1,1))
X_count   = count_vec.fit_transform(X)

TF-IDF

In [20]:
tf = TfidfTransformer(norm="l1")
X_freq_count = tf.fit_transform(X_count)

# classificação dos dados

In [21]:
models = {}
models['kNN'] = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
models['Random Forest'] = RandomForestClassifier(n_estimators=250,random_state=42)
models['Naive Bayes'] = GaussianNB()
models['MLP_1'] = MLPClassifier(hidden_layer_sizes=(30,),activation='relu')
models['MLP_2'] = MLPClassifier(hidden_layer_sizes=(30,30),activation='relu')
models['SVM'] = SVC()
models['DT'] = DecisionTreeClassifier(random_state=42,min_samples_split=10)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_freq_count,y,test_size=0.8,random_state=42,shuffle=True)

In [23]:
model_name   = []
result_acc   = []
result_f1    = []

for actual_model in models.keys(): 
    __some_model__ = models[actual_model].fit(X_train.toarray(),y_train)
    y_predict      = models[actual_model].predict(X_test.toarray())
    # score test 
    acc_scr    = accuracy_score(y_test,y_predict)
    f1_scr     = f1_score(y_test,y_predict,average='weighted')
    # savaing information 
    model_name.append(str(actual_model))
    result_acc.append(acc_scr)
    result_f1.append(f1_scr)

c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Results 

In [24]:
result = pd.DataFrame({'Model name': model_name, 'Accuracy':result_acc, 'F1-Score':result_f1})
result = result.sort_values(by='Accuracy')
result

,Model name,Accuracy,F1-Score
0,kNN,0.506361,0.355240
6,DT,0.591603,0.591192
2,Naive Bayes,0.611323,0.608609
5,SVM,0.697201,0.695872
1,Random Forest,0.725827,0.725583
3,MLP_1,0.772265,0.771449
4,MLP_2,0.775445,0.774692


# Otimize o Classificador

Loading hyperparameters

In [82]:
# Knn model
hyper_1 = {}
hyper_1 ['classifier__n_neighbors'] = [3,9,18,25]
hyper_1 ['classifier__leaf_size'] = [5,12,20]
hyper_1 ['classifier__algorithm'] = ['ball_tree','kd_tree']
hyper_1['classifier'] = [models['kNN']]
# Random Forest model
hyper_2 = {}
hyper_2 ['classifier__n_estimators'] = [10,20,30,50,100,150,200]
hyper_2 ['classifier__criterion'] = ['gini', 'entropy']
hyper_2 ['classifier__max_depth'] = [5,20,50,80,100]
hyper_2['classifier'] = [models['Random Forest']]
# Naive Bayes modedl
hyper_3 = {}
hyper_3 ['classifier__var_smoothing'] = [0.00001,0.001,0.1]
hyper_3['classifier'] = [models['Naive Bayes']]
# MLP model 1
hyper_4 = {}
hyper_4 ['classifier__hidden_layer_sizes'] = [(20,),(50,),(100,)]
hyper_4 ['classifier__activation'] = ['logistic','tanh','relu']
hyper_4 ['classifier__alpha'] = [0000.1, 00.1, 1]
hyper_4['classifier'] = [models['MLP_1']]
# MLP model 2
hyper_5 = {}
hyper_5 ['classifier__hidden_layer_sizes'] = [(20,20),(50,50),(100,100)]
hyper_5 ['classifier__activation'] = ['logistic','tanh','relu']
hyper_5 ['classifier__alpha'] = [0000.1, 00.1, 1]
hyper_5['classifier'] = [models['MLP_2']]
# SVM model 
hyper_6 = {}
hyper_6['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
hyper_6['classifier__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']
hyper_6['classifier__gamma'] = ['scale','auto']
hyper_6['classifier'] = [models['SVM']]
# Decision Tree Model
hyper_7 = {}
hyper_7['classifier__criterion'] = ['gini','entropy','log_loss']
hyper_7['classifier__splitter'] = ['best', 'random']
hyper_7['classifier__max_depth'] = [10,20,50,100]
hyper_7['classifier__min_samples_split'] = [2,5,10,15,25]
hyper_7['classifier'] = [models['DT']]


In [84]:
pipe = Pipeline([('classifier', models['kNN'])])
params = [hyper_1,hyper_2,hyper_3,hyper_4,hyper_5,hyper_6,hyper_7]

In [85]:
k_fold  = KFold(n_splits=3,random_state=42,shuffle=True)

In [86]:
gridCV  = GridSearchCV(estimator=pipe,param_grid=params,cv=k_fold,n_jobs=-1,scoring='accuracy',return_train_score=True)

In [87]:
%%time
gridCV.fit(X_train,y_train)

c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 933.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\Rícharde\AppData\Local\Programs\Python\Pytho

CPU times: total: 39.9 s
Wall time: 55min 53s


c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('classifier',
                                        KNeighborsClassifier(n_jobs=-1,
                                                             n_neighbors=3))]),
             n_jobs=-1,
             param_grid=[{'classifier': [KNeighborsClassifier(n_jobs=-1,
                                                              n_neighbors=3)],
                          'classifier__algorithm': ['ball_tree', 'kd_tree'],
                          'classifier__leaf_size': [5, 12, 20],
                          'classifier__n_neighbors': [3, 9, 1...
                          'classifier__kernel': ['linear', 'poly', 'rbf',
                                                 'sigmoid']},
                         {'classifier': [DecisionTreeClassifier(min_samples_split=10,
                                                                random_state=42)],
                          'classifier__criterion': ['gini', 'entropy',
                                                    'log_loss'],
                          'classifier__max_depth': [10, 20, 50, 100],
                          'classifier__min_samples_split': [2, 5, 10, 15, 25],
                          'classifier__splitter': ['best', 'random']}],
             return_train_score=True, scoring='accuracy')

In [88]:
# Best performing model and its corresponding hyperparameters
gridCV.best_params_

{'classifier': MLPClassifier(alpha=0.1, hidden_layer_sizes=(20,)),
 'classifier__activation': 'relu',
 'classifier__alpha': 0.1,
 'classifier__hidden_layer_sizes': (20,)}

In [89]:
gridCV.best_score_

0.7733253826277083

# Teste novos pré-processamentos e verifique se há melhora nos resultados.

In [92]:
reviews_df = pd.read_csv(r"C:\Users\Rícharde\Documents\Dell lead\Atividade 06\moviereviews.tsv", sep='\t')

Data Frame preprocessing 

In [93]:
reviews_df = reviews_df.drop_duplicates()
reviews_df=reviews_df.dropna()

In [94]:
# Label transform 
data = LabelBinarizer().fit_transform(reviews_df["label"])
reviews_df["label"] = data

In [95]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1940 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1940 non-null   int32 
 1   review  1940 non-null   object
dtypes: int32(1), object(1)
memory usage: 37.9+ KB


Nlp preprocessing:

Expand Contractions

In [96]:
def expand_contractions(sentence):
    expanded_words = []   
    for word in sentence.split():
        expanded_words.append(contractions.fix(word))  
    return ' '.join(expanded_words)

Lower Case

In [97]:
def lower_case(sentence):
    sentence = sentence.lower()
    return sentence

Clear data

In [98]:
# remove links,dots,commas,numbers 
def clean_text(instance):
    instance = instance.lower()
    instance = re.sub('\[.*?\]', ' ', instance)
    instance = re.sub('https?://\S+|www\.\S+', ' ', instance)
    instance = re.sub('<.*?>+', ' ', instance)
    instance = re.sub('[%s]' % re.escape(string.punctuation), ' ', instance)
    instance = re.sub('\n', '', instance)
    instance = re.sub('\w*\d\w*', ' ', instance)
    return instance

Remove Stopwords

In [99]:
def RemoveStopWords(instance):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instance.split() if not i in stopwords]
    return (" ".join(palavras))

Applying Lemmatization

In [100]:
wordnet_lemmatizer = WordNetLemmatizer()

def Lemmatization(instancia):
  palavras = []
  for w in instancia.split():
    palavras.append(wordnet_lemmatizer.lemmatize(w))
  return (" ".join(palavras))

Remove Extra Spaces

In [101]:
def remove_extra_spaces(instance):
    instance = re.sub(' +', ' ', instance)
    return instance

Running new preprocessing 

In [102]:
def pre_processing(df,local_data):
    df[local_data] = df[local_data].apply(lambda local_data: expand_contractions(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: lower_case(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: clean_text(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: RemoveStopWords(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: Lemmatization(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: remove_extra_spaces(local_data))
    return df 

In [103]:
reviews_df = pre_processing(reviews_df,'review')

Separando dados 

In [104]:
X = reviews_df.review
y = reviews_df.label

Vectorize

BOW

In [105]:
count_vec = CountVectorizer(ngram_range=(1,1))
X_count   = count_vec.fit_transform(X)

TF-IDF

In [106]:
tf = TfidfTransformer()
X_freq_count = tf.fit_transform(X_count)

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X_freq_count,y,test_size=0.8,random_state=42,shuffle=True)

Best Classifier test 

In [108]:
clf = MLPClassifier(alpha=0.1,hidden_layer_sizes=(20,),activation='relu')

In [109]:
clf.fit(X_train,y_train)

c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1, hidden_layer_sizes=(20,))

In [110]:
y_pred = clf.predict(X_test)

Results

In [111]:
f1_score(y_test,y_pred)

0.7895065584009994

In [112]:
accuracy_score(y_test,y_pred)

0.782860824742268

Analisando os resultados realizando como pré-processamento a retirada de Stop Words, pontuações/carcteres especiais e tokenização tivemos um F1 de 0.7302 com o melhor modelo dentre os testados (RandomForest). 

No segundo pré-processamento aplicamos a expansão das contrações, colocamos o texto em caixa baixa, fizemos a remoção de pontuações e caracteres especiais, remoção de Stop Words, Lematização e remoção de possíveis espaços extras. Com esse novo pré-processamento, tivemos como resultado de F1-Score no Random Forest o valor ded 0.7378. Podemos constar que mesmo aplicando as novas técnicas de pré-processamento descritas a melhora na classificação foi irrisória